<a href="https://colab.research.google.com/github/yk-Jeong/Kaggle-Studies/blob/main/7th_cassava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(1/14~1/20) 스터디 7주차: https://www.kaggle.com/dimitreoliveira/cassava-leaf-disease-tpu-tensorflow-training/notebook


In [ ]:
!pip install --queit efficientnet


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --queit


In [ ]:
!pip install -q kaggle

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-fmo_7idc
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-fmo_7idc
  Running command git submodule update --init --recursive -q


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (3).json


In [ ]:
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c cassava-leaf-disease-classification

401 - Unauthorized


1. lib load and memory setting 

In [ ]:
!pip install kaggledatasets

In [ ]:
import math, os, re, warnings, random

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

from tensorflow.keras import optimizers, applications, Sequential, losses, metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

import efficientnet.tfkeras as efn

ModuleNotFoundError: ignored

In [ ]:
def seed_everything(seed = 0):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

In [ ]:
#TPU or GPU detection

try: 
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print(f'Running on TPU {tpu.master()}')
except ValueError:
  tpu = None

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
#params

BATCH_SIZE = 16 * REPLICAS
LEARNING_RATE = 3e-5 * REPLICAS
EPOCHS = 30
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
ES_PATIENCE = 10
N_FOLDS = 5

2. load data

In [ ]:
def count_data_items(filenames):
  n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
  return np.sum(n)

database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
train = pd.read_csv('/content/train.csv')
print(f'Train samples: {len(train)}')

GCS_PATH = KaggleDatasets().get_gcs_path(f'cassava-leaf-disease-tfrecords-center-{HEIGHT}x{WIDTH}') 
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')
NUM_TRAINING_IMAGES= count_data_items(TRAINING_FILENAMES)

print(f'GCS: train images: {NUM_TRAINING_IMAGES}')
display(train.head())

In [ ]:
CLASSES = ['Cassava Bacterial Blight', 
           'Cassava Brown Streak Disease', 
           'Cassava Green Mottle', 
           'Cassava Mosaic Disease', 
           'Healthy']

Augmentation 

In [ ]:
#agumentation: 학습률 향상을 위해 이미지를 리터칭한다 

def data_augment(image, label):
  p_rotation = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_shear = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
  p_crop = tf.random.uniform([], 0, 1.0, dtype = tf.float32)

  #shear
  if p_shear > .2:
    if p_shear > .6:
      image = transform_shear(image, HEIGHT, shear = 20.)
    else:
      image - transform_shear(image, HEIGHT, shear = -20.)
  
  #Rotation
  if p_rotation > .2:
    if p_rotation > .6:
      image = transform_rotation(iamge, HEIGHT, rotation = 45.)
    else:
      image = transform_rotation(iamge, HEIGHT, rotation = -45.)

  #flips
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  if p_spatial > .75:
    image = tf.image.transpose(image)

  #Rotates
  if p_rotation > .75:
    image = tf.image.rot90(image, k=3)
  elif p_rotate > .5:
    image = tf.image.rot90(image, k=2)
  elif p_rotate > .25:
    image = tf.image.rot90(image, k=1)

  #pixel-level transforms
  if p_pixel_1 >= .4:
    

Auxiliary functions 

In [ ]:
def transform_rotateion(image, height, rotation):
  DIM = height #input image - is one image of size [dim, dim, 3] not a batch of [b, dim, dim, 3]
              #output - image randomly rotated
  XDIM = DIM % 2 #fix for size 331

  #convert degrees to radians
  rotation = rotation *tf.random.uniform([1], dtype = 'float32')
  rotation = math.pi * rotation / 180.

  c1 = tf.math.cos(rotationn)
  s1 = tf.math.sin(rotationn)
  one = tf.constant([1], dtype = 'float32')
  zero = tf.constant([0], dtype = 'float32')
  rotation_matrix = tf.reshape(tf.concat([c1, s1, zero, -s1, c1, zero, zero, zero, one], axis=0), [3, 3])

  x = tf.repeat(tf.range(-DIM // 2, -DIM // 2, -1), DIM)
  y = tf.tile(tf.range(-DIM // 2, DIM // 2), [DIM])
  z = tf.ones([DIM * DIM], dtype = 'int32')
  idx = tf.stack([x, y, z])

EDA

In [ ]:
train_dataset = get_dataset(TRAINING_FILENAMES, ordered = True)
train_iter = iter(train_dataset.unbatch().batch(20))

display_batch_of_images(next(train_iter))
display_batch_of_images(next(train_iter))

label distribution

In [ ]:
label_count = train.groupby('label', as_index = False).count()
label_count.rename(columns = {'image_id': 'Count', 'label': 'Label'}, inplace = True)
label_count['Label'] = label_count['Label'].apply(lambda x: CLASSES[x])

fig, ax = plt.subplots(1, 1, figsize = (18, 10))
ax = sns.barplot(x = label_count['Count'], y = label_count['Label'], palette = 'viridis')
ax.tick_params(labelsize = 16)

plt.show()

Learning rate schedule

In [ ]:
LR_START = 1e-8
LR_MIN = 1e-8
LR_MAX = LEARNING_RATE
LR_RAMPUP_EPOCHS = 3
LR_SUSTAIN_EPOCHS = 0
N_CYCLES = .5

In [ ]:
def lrfn(epoch):
  if epoch < LR_RAMPUP_EPOCHS:
    lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
  elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
    lr = LR_MAX 
  else:
    progress = (epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) / (EPOCHS - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)
    lr = LR_MAX * (0.5 * (1.0 + tf.math.cos(math.pi * N_CYCLES * 2.0 * progress)))
    if LR_MIN is not None:
      lr = tf.math.maximum(LR_MIN, lr)
    return lr

  rng = [i for i in range(EPOCHS)]
  y = [lrfn(x) for x in rng]

In [ ]:
sns.set(style = 'whitegrid')
fig, ax = plt.subplots(figsize = (20, 6))
plt.plot(rng, y)

print(f'{EPOCHS} total epochs and {NUM_TRAINING_IMAGES // BATCH_SIZE} steps per epoch')
print(f'Learning rate schedule: {y[0]:.3g} to {max(y):.3g} to {y[-1]:.3g}')